## Classification (b)

In [1]:
from neuralnet import *
from random import shuffle
import numpy as np

# Load the configuration.
config = load_config("./data")

# Create the model
#model  = Neuralnetwork(config)
# Load the data
#x_train, y_train = load_data(path="./data", mode="train") # output normalized and one hot encoding
#x_test, y_test = load_data(path="./data", mode="test")# output normalized and one hot encoding

# Use 10 examples one from each category
'''
# Data loading and spliting
X_b, y_b, X_stats = load_data(path="./data", mode="train")

X_train = []
y_train = []
class_num = list(range(10))


X_train, y_train = [], []
for idx in range(y_b.shape[0]):
    if len(class_num) == 0:
        break
    indice = onehot_decode(y_b[idx])
    
    if indice in class_num:
        X_train.append(X_b[idx])
        y_train.append(y_b[idx])
        class_num.remove(indice)
        
        
X_train = np.array(X_train)        
y_train = np.array(y_train)       
'''

X_train, y_train, X_stats = load_data(path="./data", stats=None, mode="train")

# Get 10 examples, 1 from each category.
X_sub = []
y_sub = []
for k in range(10):
    indices = y_train[:,k] == 1
    X_sub.append(X_train[indices][0])
    y_sub.append(y_train[indices][0])
print(len(X_sub), len(y_sub))
X_sub = np.array(X_sub)
y_sub = np.array(y_sub)

# Load model para
config_prob_b = {}
config_prob_b['layer_specs'] = [3072, 10, 10, 10]
config_prob_b['activation'] = 'tanh'
config_prob_b['learning_rate'] = 0.005
config_prob_b['batch_size'] = 128 
config_prob_b['epochs'] = 100  
config_prob_b['early_stop'] = True 
config_prob_b['early_stop_epoch'] = 5  
config_prob_b['L2_penalty'] = 0  
config_prob_b['momentum'] = False  
config_prob_b['momentum_gamma'] = 0.9  

model_b = Neuralnetwork(config_prob_b)
model_b.forward(X_sub, y_sub)
model_b.backward()




(50000, 32, 32, 3)
inp: (50000, 32, 32, 3)
10 10
in backward layer
size: 10
delta: (10, 10)
change: 4.440892098500626e-16
change: 4.163336342344337e-17
*x.T_shape: (10, 10)
*delta_shape: (10, 10)
*d_w_shape: (10, 10)
d_w :  -4.163336342344337e-17
d_b :  -4.163336342344337e-17
in backward layer
size: 10
delta: (10, 10)
change: -3.7803481728656942
change: -0.3780348172865692
*x.T_shape: (10, 10)
*delta_shape: (10, 10)
*d_w_shape: (10, 10)
d_w :  0.3780348172865692
d_b :  0.3780348172865692
in backward layer
size: 10
delta: (10, 10)
change: 482.8810474192792
change: 48.288104741927924
*x.T_shape: (3072, 10)
*delta_shape: (10, 10)
*d_w_shape: (3072, 10)
d_w :  -48.288104741927924
d_b :  -48.288104741927924


array([[-1.32850882e-05,  1.45903488e-05, -2.32924336e-05, ...,
        -2.46434433e-06, -1.10828988e-05,  1.08532447e-05],
       [ 3.12593139e-03,  1.13898808e-04,  5.11964177e-03, ...,
        -9.95171889e-04,  3.88315850e-03,  1.25684977e-03],
       [ 1.50207731e-12,  5.95918566e-12, -2.63289052e-12, ...,
        -3.17851575e-12,  6.89494773e-12,  6.17360722e-12],
       ...,
       [ 2.74560090e-11, -3.34223777e-11, -2.49502369e-11, ...,
         4.62296638e-12, -1.82286031e-11, -1.87014343e-12],
       [-2.16030976e-03, -8.57199430e-03,  3.78689084e-03, ...,
         4.57200758e-03, -9.91783999e-03, -8.88010572e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [2]:
# d_b d_w comparison

## Part (b) Estimation of bias weight and weight
def Num_Est_b(model, layer, eps, output_idx):
    layer.b[0][output_idx] += eps # E(w+e)
    loss_1 = model.forward(X_sub, y_sub)[1]
    layer.b[0][output_idx] -= 2*eps # E(w-e)
    loss_2 = model.forward(X_sub, y_sub)[1]
    layer.b[0][output_idx] += eps # back to normal
    print("diff:", loss_1 - loss_2)
    return (loss_1 - loss_2) / (2 * eps) # Numerical estimation for dEdW

def Num_Est_w(model, layer, eps, input_idx, output_idx):
    layer.w[input_idx][output_idx] += eps # E(w+e)
    loss_1 = model.forward(X_sub, y_sub)[1]
    layer.w[input_idx][output_idx] -= 2*eps # E(w-e)
    loss_2 = model.forward(X_sub, y_sub)[1]
    layer.w[input_idx][output_idx] += eps # back to normal
    print("diff:", loss_1 - loss_2)
    return (loss_1 - loss_2) / (2 * eps) # Numerical estimation for dEdW

print("Layers:", len(model_b.layers))
# Weights to modify:
# 1 output bias weight
# 1 hidden bias weight for each hidden layer
# 2 hidden to output weights
# 2 input to hidden weights
# Show that the grad between is within O(eps^2) of backprop weights.
db = []
dw = []
db_est = []
dw_est = []

eps = 10e-2
# Hidden Layer 1
hidden_layer_1 = model_b.layers[0]
# Hidden Layer 2
hidden_layer_2 = model_b.layers[2]
# Output Layer
output_layer = model_b.layers[4]

# 1 output bias weight
input_idx = 0
output_idx = 1
cur_layer = output_layer
db_est.append(Num_Est_b(model_b, cur_layer, eps, output_idx))
db.append(cur_layer.d_b[0][output_idx]) 
print(f"Output bias weight diff: {db_est[-1] - db[-1]:.6f}")

# 1 hidden bias weight for hidden layer 2
cur_layer = hidden_layer_2
db_est.append(Num_Est_b(model_b, cur_layer, eps, output_idx))
db.append(cur_layer.d_b[0][output_idx]) 
print(f"Hidden Layer 2 bias weight diff: {db_est[-1] - db[-1]:.6f}")

# 1 hidden bias weight for hidden layer 1
cur_layer = hidden_layer_1
db_est.append(Num_Est_b(model_b, cur_layer, eps, output_idx))
db.append(cur_layer.d_b[0][output_idx]) 
print(f"Hidden Layer 1 bias weight diff: {db_est[-1] - db[-1]:.6f}")

# 2 hidden to output weights
input_idx = 0
output_idx = 1
cur_layer = hidden_layer_2
dw_est.append(Num_Est_w(model_b, cur_layer, eps, input_idx, output_idx))
dw.append(cur_layer.d_w[input_idx][output_idx])
print(f"Hidden to output weight diff 1: {dw_est[-1] - dw[-1]:.6f}")
input_idx = 0
output_idx = 2
dw_est.append(Num_Est_w(model_b, cur_layer, eps, input_idx, output_idx))
dw.append(cur_layer.d_w[input_idx][output_idx])
print(f"Hidden to output weight diff 2: {dw_est[-1] - dw[-1]:.6f}")

# 2 input to hidden weights
input_idx = 0
output_idx = 1
cur_layer = hidden_layer_1
dw_est.append(Num_Est_w(model_b, cur_layer, eps, input_idx, output_idx))
dw.append(cur_layer.d_w[input_idx][output_idx])
print(f"Input to hidden 1 weight diff 1: {dw_est[-1] - dw[-1]:.6f}")
input_idx = 0
output_idx = 2
dw_est.append(Num_Est_w(model_b, cur_layer, eps, input_idx, output_idx))
dw.append(cur_layer.d_w[input_idx][output_idx])
print(f"Input to hidden 2 weight diff 2: {dw_est[-1] - dw[-1]:.6f}")


Layers: 5
diff: -0.009038896391754037
Output bias weight diff: 0.000050
diff: 0.022823174643701982
Hidden Layer 2 bias weight diff: 0.000205
diff: -2.0126290360522603e-05
Hidden Layer 1 bias weight diff: -0.000001
diff: -0.022948392814019947
Hidden to output weight diff 1: -0.000191
diff: -0.011598303563825407
Hidden to output weight diff 2: 0.000288
diff: -1.3374003522415023e-06
Input to hidden 1 weight diff 1: -0.000000
diff: -0.006792612416164356
Input to hidden 2 weight diff 2: -0.000310
